## Recursion Solution of the European Binomial Model

In [6]:
## First import numpy
import numpy as np
from scipy.stats import binom

## Call Payoff Function
def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

## Put Payoff Function
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

### The Simplified European Binomial Pricer

In [7]:
def euroBinomPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    for i in range(nodes):
        prob = binom.pmf(i, n, pstar)
        spotT = S * (u ** i) * (d ** (n - i))
        po = payoff(spotT, K) 
        price += po * prob
        if verbose:
            print(f"({spotT:0.4f}, {po:0.4f}, {prob:0.4f})")
        
    price *= np.exp(-r * T)
    
    return price

In [8]:
S = 41.0
K = 40.0
r = 0.08
v = 0.30
q = 0.0
T = 1.0
n = 3

putPrc = euroBinomPricer(S, K, r, v, q, T, n, putPayoff, verbose = True)
print(f"The European Put Premium: {putPrc : 0.3f}")

(26.4157, 13.5843, 0.1603)
(37.3513, 2.6487, 0.4044)
(52.8140, 0.0000, 0.3400)
(74.6781, 0.0000, 0.0953)
The European Put Premium:  2.999


### The Full Recursive Solution for the European Binomial Pricer

In [16]:
def euroBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1 - pu
    disc = np.exp(-r * h)
    
    ## Arrays to store the spot prices and option values
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
        
    for t in range((n - 1), -1, -1):
        for j in range(t + 1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j + 1])
            
            
    return Ct[0]        

In [17]:
putPrc = euroBinomPricerRecursive(S, K, r, v, q, T, n, putPayoff, verbose = False)
putPrc

2.9985071167431516